# Code Written by:
**Shweta Tiwari**
*20 Oct 2023*

## Algorithm: Floyd Steinberg

In [2]:
import time

In [1]:
!pip install --upgrade bokeh==2.4.3

In [3]:
import numpy as np
from PIL import Image
from requests import get
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import layout
from bokeh.palettes import gray

output_notebook()

# Algorithm

In [4]:
%%time
def image_dither(path, black='#000000', white='#ffffff'):
    image_rgb = read_image(path)
    image_gray = grayscale(image_rgb)
    image_bw = floyd_steinberg(image_gray)

    show(layout([[
        plot(image_gray, palette=gray(256)),
        plot(image_bw, palette=[black, white])
    ]]))

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.82 µs


In [5]:
%%time
def floyd_steinberg(image):
    image = image.copy()
    distribution = np.array([7, 3, 5, 1], dtype=float) / 16
    u = np.array([0, 1, 1, 1])
    v = np.array([1, -1, 0, 1])

    for y in range(image.shape[0] - 1):
        for x in range(image.shape[1] - 1):
            value = np.round(image[y, x])
            error = image[y, x] - value
            image[y, x] = value
            image[y + u, x + v] += error * distribution

    image[:, -1] = 1
    image[-1, :] = 1

    return image

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs


In [6]:
%%time
def grayscale(image):
    height, width, _ = image.shape

    image = np.array(image, dtype=np.float32) / 255
    image = image[:, :, 0] * .21 + image[:, :, 1] * .72 + image[:, :, 2] * .07

    return image.reshape(height, width)

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 10.3 µs


In [7]:
%%time
def read_image(path, size=400):
    if path.startswith('https://'):
        image = Image.open(get(path, stream=True).raw)
    else:
        image = Image.open(path)

    width, height = image.size
    width, height = size, int(size * height / width)
    image = image.resize((width, height), Image.ANTIALIAS)

    data = image.getdata()
    assert data.bands in [3, 4], 'RGB or RGBA image is required'

    raw = np.array(data, dtype=np.uint8)
    return raw.reshape(height, width, data.bands)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs


In [8]:
%%time
def plot(image, palette):
    y, x = image.shape

    plot = figure(x_range=(0, x), y_range=(0, y), plot_width=x, plot_height=y)
    plot.axis.visible = False
    plot.toolbar_location = None
    plot.min_border = 0
    plot.image([np.flipud(image)], x=0, y=0, dw=x, dh=y, palette=palette)

    return plot

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 8.82 µs


# Run

In [9]:
%%time
URL = lambda name: 'https://raw.githubusercontent.com/coells/100days/master/resource/day 96 - %s.jpg' % (name,)
# URL = lambda name: './resource/day 96 - %s.jpg' % (name,)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.63 µs


In [10]:
%%time
image_dither(URL('valinka'))

Output hidden; open in https://colab.research.google.com to view.

In [11]:
%%time
image_dither(URL('eagle'), white='#ffebcd')

<timed exec>:9: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.


CPU times: user 2.48 s, sys: 38.3 ms, total: 2.52 s
Wall time: 2.62 s


In [12]:
%%time
image_dither(URL('winter'), white='#f0f0ff')

<timed exec>:9: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.


CPU times: user 2.28 s, sys: 35.5 ms, total: 2.31 s
Wall time: 2.41 s


# The End